<a href="https://colab.research.google.com/github/cristhiansito123/MLDS6project/blob/entrega4/scripts/deployment/despliegue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Despliegue en MLflow

In [1]:
!pip install mlflow==2.1.0 requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 5.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 572.6/572.6 kB 26.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 3.6 MB/s eta 0:00:00
  Created wheel for databricks-cli: filename=databricks_cli-0.17.7-py3-none

In [2]:
command = """
mlflow server \
        --backend-store-uri sqlite:///tracking.db \
        --default-artifact-root file:mlruns \
        -p 5000 &
"""
get_ipython().system_raw(command) #**TODO**: 27869

In [3]:
!pip install pyngrok

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 20.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=2866e984f675790e7b49e5b4c9ff235119e5bbc69f0ff8793744f772981e97e8
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [4]:
import json
import requests
import os
import shutil
import time
import sys

import numpy as np
import pandas as pd

In [5]:
token = "2RRhiSrNz2N4QzRvzFTAH3ZfZV0_5aWRhfdGQdDRNpK7cNxni" # Agregue el token dentro de las comillas
os.environ["NGROK_TOKEN"] = token

autenticación en ngrok

In [6]:
!ngrok authtoken $NGROK_TOKEN

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


Ahora, lanzamos la conexión

In [7]:
from pyngrok import ngrok
ngrok.connect(5000, "http")

<NgrokTunnel: "https://d5e9-35-236-247-25.ngrok-free.app" -> "http://localhost:5000">

Especificamos el servidor de MLFlow

In [8]:
import mlflow
import os
import pandas as pd
from IPython.display import display

In [9]:
mlflow.set_tracking_uri("http://localhost:5000")

In [11]:
exp_id = mlflow.create_experiment(name="fraudulent_transactions", artifact_location="mlruns/")

## Llamado del modelo

In [12]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


import neccesary packages

In [13]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from imblearn.over_sampling import SMOTE
%matplotlib inline

#Carga de datos

In [14]:
metadata_path = '/content/drive/MyDrive/MLDS6project/data.csv'

metadata = pd.read_csv(metadata_path, sep=',')

In [15]:
mapeo = {'CASH_OUT': 1, 'PAYMENT': 2, 'CASH_IN': 3, 'TRANSFER': 4, 'DEBIT': 5}

# Aplicar el mapeo a la columna del DataFrame
metadata['type_numeric'] = metadata['type'].map(mapeo)

In [16]:
feature_names = ['step', 'type_numeric', 'amount', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest']
X = metadata[feature_names]
y = metadata['isFraud']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [18]:

route_model = '/content/drive/MyDrive/MLDS6project/XGBclassifier.joblib'

In [19]:
from xgboost import XGBClassifier
import joblib

model = joblib.load(route_model)

In [20]:
model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=100, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

## Importamos el modelo

In [21]:
from sklearn.metrics import balanced_accuracy_score,roc_auc_score,f1_score,average_precision_score,accuracy_score,roc_curve

In [23]:
with mlflow.start_run(
        run_name="xgboost",
        experiment_id=exp_id
        ):
    y_pred = model.predict(X_test)
    mlflow.log_metric("Accuracy", accuracy_score(y_test, y_pred))
    mlflow.log_metric("Precision", average_precision_score(y_test, y_pred))
    mlflow.log_metric("F1_score", f1_score(y_test, y_pred))
    mlflow.log_metric("Area_under_curve", roc_auc_score(y_test, y_pred))
    mlflow.xgboost.log_model(model, "model")

/usr/local/lib/python3.10/dist-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")


Despliegue del modelo

In [24]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "http://localhost:5000"

In [25]:
command = """
mlflow models serve -m 'models:/fraudulent_transactions/Production' -p 8001 --env-manager 'local' &
"""
get_ipython().system_raw(command)

In [27]:
import requests

In [54]:
input = X_train.sample(n=3)

In [55]:
input

,step,type_numeric,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest
2682611,210,2,18373.92,133.00,0.00,0.0,0.0
5812607,401,2,4479.11,67739.43,63260.32,0.0,0.0
3342479,253,2,10138.20,0.00,0.00,0.0,0.0


In [58]:
data = input.to_dict(orient='records')

# Convertir a JSON
json_data = json.dumps(data)


In [60]:
json_data

'[{"step": 210, "type_numeric": 2, "amount": 18373.92, "oldbalanceOrg": 133.0, "newbalanceOrig": 0.0, "oldbalanceDest": 0.0, "newbalanceDest": 0.0}, {"step": 401, "type_numeric": 2, "amount": 4479.11, "oldbalanceOrg": 67739.43, "newbalanceOrig": 63260.32, "oldbalanceDest": 0.0, "newbalanceDest": 0.0}, {"step": 253, "type_numeric": 2, "amount": 10138.2, "oldbalanceOrg": 0.0, "newbalanceOrig": 0.0, "oldbalanceDest": 0.0, "newbalanceDest": 0.0}]'

In [62]:
r = requests.post("http://localhost:8001/invocations",json={"inputs": json_data})
print(r.text)

ConnectionError: ignored

In [53]:
y_train.loc[6292381]

0